Download the file to be sent

TODO List:
1) make the checksum

In [1]:
!wget https://www.dropbox.com/s/g8oq6mm2j1zhkv2/test_file.txt

--2018-05-12 19:40:11--  https://www.dropbox.com/s/g8oq6mm2j1zhkv2/test_file.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.1, 2620:100:6022:1::a27d:4201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/4jboFJHYIRoqEWAu8ZiUbONR74Fgvfb1AUzXS29R2LaSIADiErb0C10MzZi1UErV/file [following]
--2018-05-12 19:40:12--  https://dl.dropboxusercontent.com/content_link/4jboFJHYIRoqEWAu8ZiUbONR74Fgvfb1AUzXS29R2LaSIADiErb0C10MzZi1UErV/file
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.66.6, 2620:100:6022:6::a27d:4206
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.66.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24576 (24K) [text/plain]
Saving to: ‘test_file.txt.5’

test_file.txt.5     100%[===================>]  24.00K  --.-KB/s    in 0.006s  

2018-05-12 19:40:13 (

In [2]:
import socket, pickle
receiverIP=socket.gethostbyname(socket.gethostname())
from socket import *
senderPort = 12000
receiverPort = 5200 

In [3]:
class Packet:
  def __init__(
  self,
  seq_num=None,
  checksum=None,
  data=None,
  type_ =1,
  act_num=None
  ):
    self.seq_num=seq_num
    self.checksum=checksum
    self.data=data
    self.type_=type_
    self.act_num=act_num
    
  def __lt__(self, other):
        return self.seq_num < other.seq_num

class ACK:
  def __init__(
  self,
  seq_num=None,
  checksum=None,
  real=None
  ):
    self.seq_num=seq_num
    self.checksum=checksum
    self.real=real
  

In [4]:
import random
'''
A function to decide whether to pass the ack/packet or not depending on the probability passed
and a randomly generated number
'''
def decision(probability):
    return random.random() < probability

In [5]:
import csv
'''
transform a file to a series of packets
@param sw : boolean whether this is stop and wait or not
@param seq_num_range : maximum value allowed for sequence number to have -1
'''
def file_to_pkt(file_name, sw, seq_num_range):
    packets = []
    f = open(file_name, 'rb')
    data = f.read(1024)
    seqNO = 0
    while data:
        if sw: #make sequence numbers 0 and 1 only in stop and wait
            packets.append(Packet(seq_num=seqNO % 2, cheacksum=0, data=data,act_num=seqNO))

        else:
            packets.append(Packet(seq_num=seqNO % seq_num_range, checksum=0, data=data,act_num=seqNO))
        data = f.read(1024)
        
        seqNO += 1
    #type 0 is the terminating packet of the file   
    packets[len(packets) - 1].type_ = 0
    return packets
  

def pkt_to_file(file_name, packets):
    f = open(file_name, 'wb')
    for i in range(len(packets)):
        f.write(packets[i].data)
    f.close()
    
def get_stats(file_name,stats_dict):
  with open(file_name, "w") as f:
    writer = csv.writer(f, delimiter="\t", lineterminator="\r\n")
    writer.writerows(stats_dict.items())
    


In [6]:
from concurrent.futures import ThreadPoolExecutor, wait, as_completed

from time import sleep
from random import randint
from threading import Thread, Lock

lock=Lock()
'''
ThreadPoolExecutor gives pool of threads specified to the timers
of packets in the window so max no of threads maybe set to window size

concurrent.futures is used to determine whether the function of the thread has completed (returned or not)
function of thread >>> time_alarm(...)

once it is completed (the function of thread) call-back function re-submit the timer if still no ack was received
call-back function >>> resend(...)
>>>> timers[number]=pool.submit(time_alarm, packet, prob, time_out,senderSocket,pool) assigns
the function time_alarm to one thread of the pool with the parameters given

 when the callback function is called >>> timers[item.seq_num].add_done_callback(resend)
once time_alarm finishs its job it passes the returned result to object called "Future" variable: [[fn]] via resend function
resend function reassigns time_alarm to the pool of threads as no ack received and packet is "RESENT"

I THINK THERE IS DEADLOCK HAPPENS IN MIDDLE OF SENDING PROCESS

if u need i can summarize the steps if u want so>>>
give me 20 minutes to scan the code then I will ask you questions
I will understand better this way

'''
def time_alarm(Packet, prob, time_out, senderSocket,pool,timers):
    time.sleep(time_out)
    print("WOW ",Packet.act_num)
    return Packet, Packet.seq_num, senderSocket, prob, time_out, pool,timers

# def resend(senderSocket, timers, prob, pool):
#     for x in as_completed(timers.values()):
#         timers[x.result()[1]]=pool.submit(time_alarm, x.result()[0], prob, time_out)
#         print("GOT--> ",x.result()[0].act_num)
#         if prob:

#             senderSocket.sendto(pickle.dumps(x.result()[0]),(receiverIP, receiverPort))
#             print("resent",x.result()[1],"actual",x.result()[1])
#     print("D.O.N.E")
'''we pass a function to this function? automatically, once time_alarm returns result it is passed 
to callback function resend'''    
def resend(fn):
    
    packet,number,senderSocket, prob, time_out, pool, timers=fn.result() # fn.result is returned result of time_alarm
    if prob:
      lock.acquire()
      try:
        senderSocket.sendto(pickle.dumps(packet),(receiverIP, receiverPort))
        print("resent",number,"actual",packet.act_num)
      finally:
        lock.release()
        
    timers[number]=pool.submit(time_alarm, packet, prob, time_out,senderSocket,pool)
    print("GOT--> ",packet.act_num)
          
            
    print("D.O.N.E")
            
            

In [7]:
# from threading import Thread, Lock, Event
# import time

# lock = Lock()

# def resend(Packet,prob,start_time,stats_,time_out,senderSocket,event):
        
#     while not event.is_set():
#       lock.acquire()
#       try:

#         if decision(prob):
#           senderSocket.sendto(pickle.dumps(Packet),(receiverIP, receiverPort))
#           t7 = time.time()
#           stats_['{0:.6f}'.format(t7 - start_time)] = \
#           'sent packet ' + str(Packet.seq_num) \
#           + ' actual: ' + str(Packet.act_num)
#           print(stats_['{0:.6f}'.format(t7 - start_time)])
#         else:
#           t7 = time.time()
#           stats_['{0:.6f}'.format(t7 - start_time)] = \
#           'dropped packet ' + str(Packet.seq_num) \
#           + ' actual: ' + str(Packet.act_num)

#       finally:
#         lock.release()
#         event.wait(time_out)
    
#     return 
        

In [8]:

''' assume they don't exist... cuz they only put to determine if an-old packet 
resent to me and i want to send ack



i'll use those only in case i already received the packet and it is resent to me again
IF THIS PACKET WAS IN THE RANGE OF MY WINDOW OR THAT IN """LAST""" WINDOW SEND ACK AND DON'T ACCEPT IT TWICE
IF PACKET LIES OUTSIDE THIS RANGE IGNORE AT ALL
I understand now xD 
lets go to the meat of the code xD
'''      
def is_in_range(num,start,step_size,range_):
  lim=[ i%range_ for i in range(start,start+step_size)]
  if num in lim:
    return True
  else:
    return False
  
def is_in_lrange(num,start,step_size,range_):
  lim=[ i%range_ for i in range(start-1,start-step_size)]
  if num in lim:
    return True
  else:
    return False
    

          

In [9]:

!ls

1.1.1.jpg				 Networks Sheet 4.odt
1.1.2.jpg				 Networks Sheet 4.pdf
1.1.3.jpg				 notes and questions
1.1.4.jpg				 recv.py
1.1.jpg					 sender.py
1.2.jpg					 Sheet_1.pdf
1.3.jpg					 Sheet_2.pdf
1.4.jpg					 sheet 3.odt
1.5.jpg					 sheet 3.pdf
1.jpg					 Sheet_4.pdf
2.jpg					 sheet 5- kareem -3356.odt
AyBatee5.ipynb				 sheet 5- kareem -3356.pdf
ch2 summary.pdf				 Sheet_5.pdf
Chapter_1_V6.1.pdf			 Socket_Prog.ipynb
Chapter_1_V6.1.ppt			 Socket_Programming2.ipynb
Chapter_2_V6.3.pdf			 Socket_Programming.ipynb
Chapter_2_V6.3.ppt			 SSPNetworksSpring_2018Introduction.ppt
Chapter_3_V6.01.pdf			 stop_and_wait.ipynb
Chapter_3_V6.0.ppt			 test_file.txt
Chapter_4_V6.2.ppt			 test_file.txt.1
Chapter_4_V7.01.pdf			 test_file.txt.2
Chapter_5_V6.01.ppt			 test_file.txt.3
Chapter_6_V6.0.ppt			 test_file.txt.4
Chapter_7_V6.0.ppt			 test_file.txt.5
Chapter_8_V6.0.ppt			 text_file.txt
Chapter_9_V6.0.ppt			 text_file.txt.1
drive-download-20180504T080804Z-001.zip  text_file.txt.2
Networks_project

In [10]:
''' should run this cell everytime we change parameters
splitted cells to divide and conquer xD'''
#@title Parameters Tuning{ form-width: "30%" }
sent_file_name = 'test_file.txt'  # @param {type:"string"}
recv_file_name = 'down_test_file_7.txt'  # @param {type:"string"}
prob = 1  # @param {type:"slider", min:0, max:1, step:0.01}
window_size = 3  # @param {type:"integer"}
seq_num_range = 6  # @param {type:"integer"}
time_out = 0.1 # @param {type:"number"}

import heapq
import os
import time
#from google.colab import files
import threading
from threading import Thread

#may move this before the function call 
start_time = time.time()

In [11]:
'''I THINK WE FIXED THE PROBLEMS :D'''
''' CAPTAIN KAREEM RERUN THE FILE WITH DIFFERENT PARAMETERS'''
'''ok'''
def recv_child():
    stats_ = {}
    #UDP socket
    receiverSocket = socket(AF_INET, SOCK_DGRAM)
    #reuse address when done with it
    receiverSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    receiverSocket.bind(('', receiverPort))

    print ('--The receiver is ready to receive--\n')

    packets = []
    window = []
    received = []
    recv_base = 0
    flag_end = False
#   receiving while loop, message is a packet of the file sent
    while True:

        (message, senderAddress) = receiverSocket.recvfrom(4096)
        #print("GOT SOMETHING")
        message = pickle.loads(message)
        #time instance of receiving the message
        #t6 = time.time()
        #last packet
        if message.type_ == 0:
            flag_end = True
        #if packet received is within current window
        if is_in_lrange(message.seq_num, recv_base, window_size,
                        seq_num_range) or is_in_range(message.seq_num,
                recv_base, window_size, seq_num_range):
            #if packet is not dropped
            if decision(prob):
                #time.sleep(0.004)
                receiverSocket.sendto(pickle.dumps(ACK(message.seq_num,
                        0, message.act_num)), senderAddress)
                t6 = time.time()
                stats_['{0:.6f}'.format(t6 - start_time)] = \
                    'receiver sent ack for packet ' \
                    + str(message.seq_num) + ' actual: ' \
                    + str(message.act_num)
                print (stats_['{0:.6f}'.format(t6 - start_time)])
            #else if packet is dropped
            else:
                t6 = time.time()
                stats_['{0:.6f}'.format(t6 - start_time)] = \
                    'receiver dropped ack for packet ' \
                    + str(message.seq_num) + ' actual: ' \
                    + str(message.act_num)

        if message.seq_num in received:
            continue
                   
        
        ''' MAY BE A MISTAKE YES '''
        '''this can produce an error: a logical one 
        if seq no = 3 and seq range =10 and window_size=5 then result is abs(-2)=2
        which is misleading for packet with seq=3 >>>> wait...'''
        ''' wait for edit>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>'''
#         if abs(message.seq_num % seq_num_range - window_size) \
#             in received:
#             received.remove(abs(message.seq_num % seq_num_range
#                             - window_size))

        if message.seq_num<window_size:
          if (message.seq_num+seq_num_range- window_size) in received:
            received.remove(message.seq_num+seq_num_range- window_size)
        else:
          if (message.seq_num-window_size) in received:
            received.remove(message.seq_num-window_size)

        received.append(message.seq_num)

        if recv_base == message.seq_num:
            window.append(message)
            for (i, j) in zip(sorted(window)[0::1], sorted(window)[1::1]):
                if j.seq_num == (i.seq_num + 1) % seq_num_range \
                    or j.seq_num == i.seq_num + 1:
                    recv_base = j.seq_num
                    packets.append(i)
                    window.remove(i)
                   # print (recv_base)
                else:
                    recv_base = (i.seq_num + 1) % seq_num_range
                    packets.append(i)
                    window.remove(i)
                    #print (recv_base)
                    break

            if len(window) == 1 and recv_base == window[0].seq_num:
                recv_base = (window[0].seq_num + 1) % seq_num_range
                packets.append(window[0])
                window.remove(window[0])
                #print (recv_base)

            
        else:
            window.append(message)
            
        if packets:
          if flag_end:
            time.sleep(0.1)

            print ('--Whole file received successfully. File is ready to download--\n')

                # get_stats('receiver_stats.txt', stats_)

            pkt_to_file(recv_file_name, packets)
            break




In [12]:
# down_test_file_7 we can download it using last cell in notebook
# can u do me a favor and make checksum 
# also we need to run prob. test with different paramters
#I only tried (selective repeat with current parameters) and file is well received but the download doesnt work xD-----------> USE ANOTHER BROWSER
# wait sec....
# run using jupyter notebook or use http://jupyter.org/try the file will be found in the directory your notebook is based

In [13]:
#Sender
def send_parent():
    #sender process
    newpid = os.fork()
    #if in child
    if newpid == 0:
        recv_child()
    else:#parent
        stats_ = {}
        time.sleep(1)
        packets = file_to_pkt(file_name=sent_file_name, sw=False,
                              seq_num_range=seq_num_range)

        senderSocket = socket(AF_INET, SOCK_DGRAM)
        senderSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
        senderSocket.bind(('', senderPort))

        # ## initial setting of window
        print("length of packets:",len(packets))
        base = 0
        index = 0
        step = 0
        window = []
        timers = {}
        pool = ThreadPoolExecutor(window_size)
        not_acked = []
        acked_num = 0
#         t=Thread(target=resend,args=(senderSocket, timers, prob, pool))
#         t.start()
#append and variable change orignal if ERROR
        if index + window_size < len(packets):
            window = packets[index:(index + window_size)]
        else:
            window = packets[index:len(packets)]

        for i in range(len(window)):
            #for each packet in window create an event object and  a timer/thread
            #key of these dics to get these object is packet sequence number
            #senderSocket.sendto(pickle.dumps(window[i]),(receiverIP, receiverPort))
            timers[window[i].seq_num] = pool.submit(time_alarm,window[i],prob,time_out,senderSocket,pool,timers)
            timers[window[i].seq_num].add_done_callback(resend)
            not_acked.append(window[i].seq_num)

        while True:

            reply = None
            (reply, receiverAddress) = senderSocket.recvfrom(4096)
            #time.sleep(0.001)
            reply = pickle.loads(reply)
            print("ack seq_num:",reply.seq_num,reply.real,"REC. ACK")
            if isinstance(reply,ACK):
                #time.sleep(0.0001)
                

                if (reply.seq_num in timers.keys()):
                    time.sleep(time_out)
                    del timers[reply.seq_num]
                    try:
                      not_acked.remove(reply.seq_num)
                    except ValueError:
                      print(not_acked,reply.seq_num)

                    acked_num += 1

                    if acked_num == len(packets):
                        break

                    if base == reply.seq_num:
                        if len(not_acked)==0:
                            step=window_size
                        elif abs(base - heapq.nsmallest(1, not_acked)[0]) > window_size - 1:
                            step = abs(seq_num_range % base - heapq.nsmallest(1, not_acked)[0])
                        elif abs(base - heapq.nsmallest(1, not_acked)[0]) <= window_size - 1:
                            step = abs(base - heapq.nsmallest(1, not_acked)[0])

                        index+=step

                        if index + window_size < len(packets):
                            
                            window = packets[index:index + window_size]
 
                        else:

                            window = packets[index:len(packets)]

                        

                        base= window[0].seq_num
                        print(window[0].act_num," ---- ",window[len(window)-1].act_num)

                        for item in window[-step:]:
                            #senderSocket.sendto(pickle.dumps(item),(receiverIP, receiverPort))
                            timers[item.seq_num]= pool.submit(time_alarm,item,prob,time_out,senderSocket,pool,timers)
                            timers[item.seq_num].add_done_callback(resend)
                            not_acked.append(item.seq_num)



        senderSocket.close()






In [14]:
#function call
send_parent()

--The receiver is ready to receive--

receiver sent ack for packet 0 actual: 0
receiver sent ack for packet 2 actual: 2
receiver sent ack for packet 1 actual: 1
length of packets: 24
WOW  0
resent 0WOW   actual1ack seq_num: 
 WOW 0resent0 
  2GOT--> 10
   resent0actualREC. ACK 
 
2D.O.N.E1 

actual GOT--> 2 
1GOT--> 
 D.O.N.E2

D.O.N.E
receiver sent ack for packet 3 actual: 3
1  ----  3
ack seq_num: 2 2 REC. ACK
WOW  ack seq_num:3 
1resent  13  REC. ACKactual
 3
GOT-->  3
D.O.N.E
receiver sent ack for packet 5 actual: 5
receiver sent ack for packet 4 actual: 4
3  ----  5
ack seq_num: 3 3 REC. ACK
4  ----  6
WOW WOW  5
 4
ack seq_num: 5 5 REC. ACK
resent 5 actual 5
GOT-->  5
D.O.N.E
resent 4 actual 4
GOT-->  4
D.O.N.E
receiver sent ack for packet 0 actual: 6
WOW ack seq_num: 4  64
 resentREC. ACK 
0 actual 6
GOT-->  6
D.O.N.E
6  ----  8
ack seq_num: 0 6 REC. ACK
receiver sent ack for packet 2 actual: 8
receiver sent ack for packet 1 actual: 7
receiver sent ack for packet 3 actual: 9
WOW

In [15]:
'''الله أكبر :D'''
!ls

1.1.1.jpg				 Networks-sheet2-kareem-3356.odt
1.1.2.jpg				 Networks Sheet 4.odt
1.1.3.jpg				 Networks Sheet 4.pdf
1.1.4.jpg				 notes and questions
1.1.jpg					 recv.py
1.2.jpg					 sender.py
1.3.jpg					 Sheet_1.pdf
1.4.jpg					 Sheet_2.pdf
1.5.jpg					 sheet 3.odt
1.jpg					 sheet 3.pdf
2.jpg					 Sheet_4.pdf
AyBatee5.ipynb				 sheet 5- kareem -3356.odt
ch2 summary.pdf				 sheet 5- kareem -3356.pdf
Chapter_1_V6.1.pdf			 Sheet_5.pdf
Chapter_1_V6.1.ppt			 Socket_Prog.ipynb
Chapter_2_V6.3.pdf			 Socket_Programming2.ipynb
Chapter_2_V6.3.ppt			 Socket_Programming.ipynb
Chapter_3_V6.01.pdf			 SSPNetworksSpring_2018Introduction.ppt
Chapter_3_V6.0.ppt			 stop_and_wait.ipynb
Chapter_4_V6.2.ppt			 test_file.txt
Chapter_4_V7.01.pdf			 test_file.txt.1
Chapter_5_V6.01.ppt			 test_file.txt.2
Chapter_6_V6.0.ppt			 test_file.txt.3
Chapter_7_V6.0.ppt			 test_file.txt.4
Chapter_8_V6.0.ppt			 test_file.txt.5
Chapter_9_V6.0.ppt			 text_file.txt
down_test_file_7.txt			 text_file.txt.1
drive-download-20

## dadaaaa

In [16]:
# sent_file_name = 'test_file.txt'  # @param {type:"string"}
# recv_file_name = 'down_test_file_8.txt'  # @param {type:"string"}
# prob = 1  # @param {type:"slider", min:0, max:1, step:0.01}
# window_size = 7  # @param {type:"integer"}
# seq_num_range = 14  # @param {type:"integer"}
# time_out = 0.01  # @param {type:"number"}

# import heapq
# import os
# import time
# #from google.colab import files
# import threading
# from threading import Thread

# start_time = time.time()


# def recv_child():
#     stats_ = {}
#     receiverSocket = socket(AF_INET, SOCK_DGRAM)
#     receiverSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
#     receiverSocket.bind(('', receiverPort))

#     print ('--The receiver is ready to receive--\n')

#     packets = []
#     window = []
#     received = []
#     recv_base = 0
#     flag_end = False

#     while True:

#         (message, senderAddress) = receiverSocket.recvfrom(4096)
#         message = pickle.loads(message)
#         t6 = time.time()
#         if message.type_ == 0:
#             flag_end = True
#         if is_in_lrange(message.seq_num, recv_base, window_size,
#                         seq_num_range) or is_in_range(message.seq_num,
#                 recv_base, window_size, seq_num_range):
#             if decision(prob):
#                 time.sleep(0.004)
#                 receiverSocket.sendto(pickle.dumps(ACK(message.seq_num,
#                         0, message.act_num)), senderAddress)
#                 t6 = time.time()
#                 stats_['{0:.6f}'.format(t6 - start_time)] = \
#                     'receiver sent ack for packet ' \
#                     + str(message.seq_num) + ' actual: ' \
#                     + str(message.act_num)
#                 print (stats_['{0:.6f}'.format(t6 - start_time)])
#             else:

#                 stats_['{0:.6f}'.format(t6 - start_time)] = \
#                     'receiver dropped ack for packet ' \
#                     + str(message.seq_num) + ' actual: ' \
#                     + str(message.act_num)

#         if message.seq_num in received:
#             continue
#         if abs(message.seq_num % seq_num_range - window_size) \
#             in received:
#             received.remove(abs(message.seq_num % seq_num_range
#                             - window_size))

#         received.append(message.seq_num)

#         if recv_base == message.seq_num:
#             window.append(message)
#             for (i, j) in zip(sorted(window)[0::1], sorted(window)[1::1]):
#                 if j.seq_num == (i.seq_num + 1) % seq_num_range \
#                     or j.seq_num == i.seq_num + 1:
#                     recv_base = j.seq_num
#                     packets.append(i)
#                     window.remove(i)
#                     print (recv_base)
#                 else:
#                     recv_base = (i.seq_num + 1) % seq_num_range
#                     packets.append(i)
#                     window.remove(i)
#                     print (recv_base)
#                     break

#             if len(window) == 1 and recv_base == window[0].seq_num:
#                 recv_base = (window[0].seq_num + 1) % seq_num_range
#                 packets.append(window[0])
#                 window.remove(window[0])
#                 print (recv_base)

#             if packets:
#                 if flag_end:
#                     time.sleep(0.1)

#                     print ('--Whole file received successfully. File is ready to download--\n')

#                 # get_stats('receiver_stats.txt', stats_)

#                     pkt_to_file(recv_file_name, packets)
#         else:
#             window.append(message)


# #Sender
# def send_parent():
#     #sender process
#     newpid = os.fork()
#     #if in child
#     if newpid == 0:
#         recv_child()
#     else:#parent
#         stats_ = {}
#         time.sleep(3)
#         packets = file_to_pkt(file_name=sent_file_name, sw=False,
#                               seq_num_range=seq_num_range)

#         senderSocket = socket(AF_INET, SOCK_DGRAM)
#         senderSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
#         senderSocket.bind(('', senderPort))

#         # ## initial setting of window

#         base = 0
#         index = 0
#         step = 0
#         window = []
#         timers = {}
#         events = {}
#         not_acked = []
#         acked_num = 0
        
# #append and variable change orignal if ERROR
#         if index + window_size < len(packets):
#             print(len(window),"before")
#             window = packets[index:(index + window_size)]
#             print(len(window),"after")
#         else:
#             print(len(window),"before")
#             window = packets[index:len(packets)]
#             print(len(window),"after")

#         for i in range(len(window)):
#             #for each packet in window create an event object and  a timer/thread
#             #key of these dics to get these object is packet sequence number
#             events[window[i].seq_num] = threading.Event()
#             timers[window[i].seq_num] = Thread(target=resend, args=(window[i],
#                 prob,
#                 start_time,
#                 stats_,
#                 time_out,
#                 senderSocket,
#                 events[window[i].seq_num],
#                 ))
#             #start timer of packet
#             not_acked.append(window[i].seq_num)
#             timers[window[i].seq_num].start()

#         while True:

#             reply = None
#             (reply, receiverAddress) = senderSocket.recvfrom(4096)
#             #time.sleep(0.001)
#             reply = pickle.loads(reply)
#             print("ack seq_num:",reply.seq_num,reply.real,"REC. ACK")
#             if isinstance(reply,ACK):
                
#                 if (reply.seq_num in timers.keys()) and (reply.seq_num in events.keys()):
#                     events[reply.seq_num].set()
#                     #time.sleep(0.0001)
#                     del timers[reply.seq_num]
#                     del events[reply.seq_num]

#                     not_acked.remove(reply.seq_num)

#                     acked_num += 1

#                     if acked_num == len(packets):
#                         break

#                     if base == reply.seq_num:
#                         if len(not_acked)==0:
#                             step=window_size
#                         elif abs(base - heapq.nsmallest(1, not_acked)[0]) > window_size - 1:
#                             step = abs(seq_num_range % base - heapq.nsmallest(1, not_acked)[0])
#                         elif abs(base - heapq.nsmallest(1, not_acked)[0]) <= window_size - 1:
#                             step = abs(base - heapq.nsmallest(1, not_acked)[0])

#                         index+=step

#                         if index + window_size < len(packets):
#                             print(len(window),"before")
#                             print(index,index+window_size)
#                             window = packets[index:index + window_size]
#                             print (len(window), 'if')
#                         else:
#                             print(len(window),"before")
#                             window = packets[index:len(packets)]
#                             print (len(window), 'else')

#                         base= window[0].seq_num


#                         for item in window[-step:]:
#                             events[item.seq_num] = threading.Event()
#                             timers[item.seq_num] =Thread(target=resend, args=(
#                                 item,
#                                 prob,
#                                 start_time,
#                                 stats_,
#                                 time_out,
#                                 senderSocket,
#                                 events[item.seq_num],
#                                 ))
#                             timers[item.seq_num].start()
#                             not_acked.append(item.seq_num)



#     senderSocket.close()



# send_parent()


## TO DOWNLOAD THE RECEIVED FILE

In [17]:
from google.colab import files
files.download(recv_file_name)

ModuleNotFoundError: No module named 'google.colab'